In [1]:
!nvidia-smi

Tue Jul  2 22:32:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.23                 Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   41C    P8              8W /  280W |     312MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay,confusion_matrix,precision_score,recall_score,roc_curve,roc_auc_score,balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import datetime
from dataclasses import dataclass
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from scipy.ndimage import gaussian_filter1d
from itertools import combinations as comb
from pyts.image import RecurrencePlot
import PIL
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from unidecode import unidecode

In [3]:
X_train_feat_norm_agg = pd.read_csv('X_train_feat_norm_agg.csv')
y_train_feat_norm_agg = pd.read_csv('y_train_feat_norm_agg.csv')
X_test_feat_norm_agg = pd.read_csv('X_test_feat_norm_agg.csv')
y_test_feat_norm_agg = pd.read_csv('y_test_feat_norm_agg.csv')

In [4]:
print(X_train_feat_norm_agg.shape)
print(y_train_feat_norm_agg.shape)
print(X_test_feat_norm_agg.shape)
print(y_test_feat_norm_agg.shape)

(2312, 12)
(2312, 1)
(1810, 12)
(1810, 1)


In [5]:
print(X_train_feat_norm_agg.columns)

Index(['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ', 'AccMagnitude',
       'GyroMagnitude', 'JerkX', 'JerkY', 'JerkZ', 'JerkMagnitude'],
      dtype='object')


In [6]:
# ['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']

In [7]:
X_train = X_train_feat_norm_agg[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
X_test = X_test_feat_norm_agg[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
print(X_train.shape)
print(X_test.shape)

(2312, 6)
(1810, 6)


In [8]:
y_train = y_train_feat_norm_agg
y_test =  y_test_feat_norm_agg

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2312, 6)
(2312, 1)
(1810, 6)
(1810, 1)


In [10]:
from sklearn.linear_model import LogisticRegression
Logit = LogisticRegression(C=1e2,
                          multi_class='ovr',
                          random_state=17,
                          max_iter=200
                          )
Logit.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100.0, max_iter=200, multi_class='ovr', random_state=17)

In [11]:
y_test_pred = Logit.predict(X_test)
y_test_pred_proba = Logit.predict_proba(X_test)
print(y_test_pred[:5])
print(y_test_pred_proba[:5])

[1 1 1 1 1]
[[0.53830977 0.46169023]
 [0.81696691 0.18303309]
 [0.81177037 0.18822963]
 [0.81095523 0.18904477]
 [0.56515406 0.43484594]]


In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.569060773480663
[[358 455]
 [325 672]]
              precision    recall  f1-score   support

           1     0.5242    0.4403    0.4786       813
           2     0.5963    0.6740    0.6328       997

    accuracy                         0.5691      1810
   macro avg     0.5602    0.5572    0.5557      1810
weighted avg     0.5639    0.5691    0.5635      1810



In [13]:
### 3. MLP Classifier

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=17, max_iter=1000).fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
y_test_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.598342541436464
[[394 419]
 [308 689]]
              precision    recall  f1-score   support

           1     0.5613    0.4846    0.5201       813
           2     0.6218    0.6911    0.6546       997

    accuracy                         0.5983      1810
   macro avg     0.5915    0.5878    0.5874      1810
weighted avg     0.5946    0.5983    0.5942      1810



In [15]:
### 4. KNN

from sklearn.neighbors import KNeighborsClassifier

# 5개 근접한 이웃, 거리측정기준 : 유클리드
knn = KNeighborsClassifier(n_neighbors=5, p=2)

# Model Fitting 과정
knn.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [16]:
y_test_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5602209944751381
[[406 407]
 [389 608]]
              precision    recall  f1-score   support

           1     0.5107    0.4994    0.5050       813
           2     0.5990    0.6098    0.6044       997

    accuracy                         0.5602      1810
   macro avg     0.5549    0.5546    0.5547      1810
weighted avg     0.5593    0.5602    0.5597      1810



In [17]:
### 5. Random Forest

from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=5, random_state=17)
rf_model.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(n_estimators=5, random_state=17)

In [18]:
y_test_pred = rf_model.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5552486187845304
[[425 388]
 [417 580]]
              precision    recall  f1-score   support

           1     0.5048    0.5228    0.5136       813
           2     0.5992    0.5817    0.5903       997

    accuracy                         0.5552      1810
   macro avg     0.5520    0.5523    0.5520      1810
weighted avg     0.5568    0.5552    0.5559      1810



In [19]:
### 6. Decision Tree
from sklearn.tree import DecisionTreeClassifier
DT_MODEL= DecisionTreeClassifier(random_state=17)
DT_MODEL.fit(X_train, y_train)

DecisionTreeClassifier(random_state=17)

In [20]:
y_test_pred = DT_MODEL.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5370165745856353
[[445 368]
 [470 527]]
              precision    recall  f1-score   support

           1     0.4863    0.5474    0.5150       813
           2     0.5888    0.5286    0.5571       997

    accuracy                         0.5370      1810
   macro avg     0.5376    0.5380    0.5361      1810
weighted avg     0.5428    0.5370    0.5382      1810



In [21]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

In [22]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb = le.fit_transform(y_train)
le = LabelEncoder()
y_test_xgb = le.fit_transform(y_test)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
xgb_class = XGBClassifier()
xgb_class.fit(X_train, y_train_xgb)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [24]:
y_test_pred = xgb_class.predict(X_test)
print(accuracy_score(y_test_xgb, y_test_pred))
print(confusion_matrix(y_test_xgb, y_test_pred))
print(classification_report(y_test_xgb, y_test_pred, digits=4))

0.5453038674033149
[[417 396]
 [427 570]]
              precision    recall  f1-score   support

           0     0.4941    0.5129    0.5033       813
           1     0.5901    0.5717    0.5807       997

    accuracy                         0.5453      1810
   macro avg     0.5421    0.5423    0.5420      1810
weighted avg     0.5469    0.5453    0.5460      1810



In [25]:
### 8. LightGBM
from lightgbm import LGBMClassifier
lgbm_class = LGBMClassifier()
lgbm_class.fit(X_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier()

In [26]:
y_test_pred =lgbm_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5668508287292817
[[419 394]
 [390 607]]
              precision    recall  f1-score   support

           1     0.5179    0.5154    0.5166       813
           2     0.6064    0.6088    0.6076       997

    accuracy                         0.5669      1810
   macro avg     0.5622    0.5621    0.5621      1810
weighted avg     0.5667    0.5669    0.5668      1810



In [28]:
# # LightGBM Classifier 초기화
# lgbm = LGBMClassifier()

# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'num_leaves': [31, 50, 70],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'n_estimators': [100, 200, 500],
#     'max_depth': [5, 10, 15]
# }

# # GridSearchCV 설정
# grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# # 최적의 하이퍼파라미터 찾기
# grid_search.fit(X_train, y_train)

In [29]:
from catboost import CatBoostClassifier
cat_class = CatBoostClassifier()
cat_class.fit(X_train, y_train)

Learning rate set to 0.014735
0:	learn: 0.6921474	total: 142ms	remaining: 2m 22s
1:	learn: 0.6909873	total: 144ms	remaining: 1m 11s
2:	learn: 0.6896911	total: 152ms	remaining: 50.7s
3:	learn: 0.6885070	total: 156ms	remaining: 38.9s
4:	learn: 0.6873205	total: 160ms	remaining: 31.8s
5:	learn: 0.6862483	total: 168ms	remaining: 27.8s
6:	learn: 0.6852055	total: 172ms	remaining: 24.3s
7:	learn: 0.6841851	total: 179ms	remaining: 22.2s
8:	learn: 0.6833383	total: 182ms	remaining: 20s
9:	learn: 0.6824050	total: 184ms	remaining: 18.2s
10:	learn: 0.6815001	total: 186ms	remaining: 16.7s
11:	learn: 0.6807355	total: 187ms	remaining: 15.4s
12:	learn: 0.6798106	total: 194ms	remaining: 14.7s
13:	learn: 0.6789492	total: 197ms	remaining: 13.8s
14:	learn: 0.6780889	total: 198ms	remaining: 13s
15:	learn: 0.6771836	total: 201ms	remaining: 12.4s
16:	learn: 0.6762930	total: 204ms	remaining: 11.8s
17:	learn: 0.6754248	total: 208ms	remaining: 11.3s
18:	learn: 0.6746325	total: 211ms	remaining: 10.9s
19:	learn: 0.

In [30]:
y_test_pred = cat_class.predict(X_test)
print(accuracy_score(y_test, y_test_pred))
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred, digits=4))

0.5906077348066299
[[394 419]
 [322 675]]
              precision    recall  f1-score   support

           1     0.5503    0.4846    0.5154       813
           2     0.6170    0.6770    0.6456       997

    accuracy                         0.5906      1810
   macro avg     0.5836    0.5808    0.5805      1810
weighted avg     0.5870    0.5906    0.5871      1810

